# Modulation and Coding Performance

SpaceLink includes a registry of modulation and coding combinations. Modes and the
associated error rate performance data is stored in a set of YAML files with a simple
schema such that new modes can be easily added.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

%matplotlib widget

import astropy.units as u
from spacelink.phy.registry import Registry
from spacelink.phy.performance import ErrorMetric

The error rate performance of different modulation and coding options can be compared
in this interactive plot.

In [ ]:
modcod_registry = Registry()
modcod_registry.load()

mode_ids = sorted(modcod_registry.modes.keys())

ebn0 = np.linspace(-2, 10, 100) * u.dB
fig, ax = plt.subplots(1, 1, figsize=(20, 8))
fig.subplots_adjust(right=0.45)

# Define line styles for each error metric
error_metric_styles = {
    ErrorMetric.BER: {'linestyle': '-', 'label_suffix': ' (BER)'},
    ErrorMetric.WER: {'linestyle': '--', 'label_suffix': ' (WER)'},
    ErrorMetric.FER: {'linestyle': '-.', 'label_suffix': ' (FER)'}
}

@widgets.interact(
    modcod_ids=widgets.SelectMultiple(
        options=mode_ids,
        value=(mode_ids[0],),
        description="Modes:",
        rows=min(12, len(mode_ids)),
        layout=widgets.Layout(width="50%", height="150px"),
        style={"description_width": "auto"},
    ),
    error_types=widgets.SelectMultiple(
        options=[('Bit Error Rate (BER)', ErrorMetric.BER),
                ('Codeword Error Rate (WER)', ErrorMetric.WER),
                ('Frame Error Rate (FER)', ErrorMetric.FER)],
        value=(ErrorMetric.BER, ErrorMetric.WER),
        description="Error Types:",
        rows=3,
        layout=widgets.Layout(width="50%", height="80px"),
        style={"description_width": "auto"},
    )
)
def plot_modcods(modcod_ids, error_types):
    ax.clear()

    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    
    # Track legend entries
    legend_handles = []
    legend_labels = []

    for idx, mode_id in enumerate(modcod_ids):
        color = colors[idx % len(colors)]
        
        for error_metric in error_types:
            try:
                modcod_perf = modcod_registry.get_performance(mode_id, error_metric)
                error_rate = modcod_perf.ebn0_to_error_rate(ebn0)
                
                style = error_metric_styles[error_metric]
                line_label = mode_id + style['label_suffix']
                
                line, = ax.semilogy(ebn0, error_rate, 
                                   label=line_label, 
                                   color=color, 
                                   linestyle=style['linestyle'])
                legend_handles.append(line)
                legend_labels.append(line_label)
                
            except KeyError:
                # Performance data not available for this mode/metric combination
                continue

    ax.set_xlabel("$E_b/N_0$ (dB)")
    ax.set_ylabel("Error Rate")
    ax.grid(True)

    if legend_handles:
        ax.legend(
            legend_handles,
            legend_labels,
            loc="upper left",
            bbox_to_anchor=(1.02, 1.0),
            borderaxespad=0.0,
            title="Modes & Metrics",
            frameon=True,
        )